### Basic code for 2 Hidden Layer Netowork for MNIST sample set 

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tqdm


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Data Locations
train_path = './data/mnist_example/digitstrain.csv'
val_path = './data/mnist_example/digitsvalid.csv'

In [3]:

train_data = pd.read_csv(train_path)
X_train = np.asarray(train_data.values[:,:784])
Y_train = np.asarray(train_data.values[:,784])
print(Y_train.shape)

(2999,)


In [4]:
n_inputs = 784
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10


## Step 1: Making the TF Graph

In [5]:
X = tf.placeholder(tf.float32, shape = [None, n_inputs], name = "X")
y = tf.placeholder(tf.int64, shape = [None], name = "y")

In [6]:
#Method-1 : Create your own funtion
# def neuron_layer(X, n_neurons, name, activation = None):
#     with tf.name_scope(name):
#         n_inputs = int(X.get_shape()[1])
#         stddev = 2 / np.sqrt(n_inputs + n_neurons)
#         init = tf.truncated_normal((n_inputs, n_neurons), stddev = stddev)
#         W = tf.Variable(init, name = "kernel")
#         b = tf.Variable(tf.zeros([n_neurons]), name = "bias")
#         Z = tf.matmul(X,W) +b
#         if activation == None:
#             return Z
#         else:
#             return activation(Z)


# with tf.name_scope("dnn"):
    
#     hidden1 = neuron_layer(X, n_hidden1, name = "hidden1", activation = tf.nn.relu)
#     hidden2 = neuron_layer(hidden1, n_hidden2, name= "hidden2", activation = tf.nn.relu)
#     logits = neuron_layer(hidden2, n_outputs, name = "outputs")

## Leaky Relu : A much better varient of relu, although elu is the best 

In [7]:
def leaky_relu(z, name= None):
    return tf.maximum(0.01*z, z, name = name) # alpha = 0.01



In [8]:
#Method -2: Use tf.layers.dense.

dropout_rate = 0.9
training = tf.placeholder_with_default(True, shape = (), name = "training")

with tf.name_scope("dnn2"):
    #he_init = tf.contrib.layers.variance_scaling_initializer() For He initializer of weights 
    #hidden1 = tf.layers.dense(X, n_hidden1, name = "hidden1", activation = tf.nn.elu, kernel_initializer= he_init)
    
    hidden1 = tf.layers.dense(X, n_hidden1, name = "hidden1", activation =tf.nn.relu)
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)
    hidden2 = tf.layers.dense(hidden1_drop, n_hidden2, name = "hidden2", activation = tf.nn.relu)
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training)

    logits = tf.layers.dense(hidden2, n_outputs, name= "logits", activation=tf.nn.elu)

In [9]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y,
                                                             logits = logits)
    loss = tf.reduce_mean(xentropy, name = "loss")

In [10]:
learning_rate = 0.01
threshold = 1.0 # For Gradient clipping 

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate= learning_rate)
    grads_vars = optimizer.compute_gradients(loss)
    capped_grads = [(tf.clip_by_value(grad, -threshold, threshold), var) for grad, var in grads_vars] # Gradient Clipping 
    
    training_op = optimizer.apply_gradients(capped_grads)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()


    

## Step 2: Execution of TF

In [11]:
batch_size = 100
val_data = pd.read_csv(val_path)
test = val_data.values 
train_data = train_data.values
np.random.shuffle(train_data)
with tf.Session() as sess:
    init.run()
    accs = []
    for epoch in tqdm(range(100)):
        np.random.shuffle(train_data)
        ind = 0
        while ind < train_data.shape[0]:
            X_batch = train_data[ind:ind+batch_size, :784]
            y_batch = train_data[ind:ind+batch_size,784]
            sess.run(training_op, feed_dict = {X : X_batch, y : y_batch})
            ind += batch_size
        accs.append(accuracy.eval(feed_dict = {X : test[:,:784], y : test[:,784], training: False}))
    for i in range(len(accs)):
        print("Epoch : {!r} , Validation Accuracy : {!r}".format(i, accs[i]))
        #print ( "" % epoch, acc_val)
    save_path = saver.save(sess, './trained_models/MLP.ckpt')

TypeError: 'module' object is not callable

## Freezing Lower layers


In [ ]:
hidden_state_stop = tf.stop_gradient(idden2)